# Figures

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.rcParams["font.family"] = "DejaVu Sans"

## Bayes' Theorem Visualization

In [ ]:
import scipy.stats as stats

In [ ]:
x, y = np.meshgrid(np.linspace(-1, 1, 201), np.linspace(-1, 1, 201))
pos = np.dstack([x, y])

n = 81
xx = np.empty([n, n], dtype=float)
yy = np.empty([n, n], dtype=float)
prior = np.empty([n, n], dtype=float)
likelihood = np.empty([n, n], dtype=float)
for i, x in enumerate(np.linspace(-1, 1, n)):
    for j, y in enumerate(np.linspace(-1, 1, n)):
        xx[i,j] = x
        yy[i,j] = y
        prior[i,j] = stats.multivariate_normal([0., 0.], [[0.2, 0.], [0., 0.2]]).pdf([x, y])
        likelihood[i,j] = stats.multivariate_normal([0.1*x - 0.3, -0.3*y], [[0.6, 0.58], [0.58, 0.6]]).pdf([x, y])
posterior = likelihood*prior

prior = prior/np.max(prior)
likelihood = likelihood/np.max(likelihood)
posterior = posterior/np.max(posterior)

levels = 7
cmap = plt.get_cmap("Greys", levels)
norm = plt.Normalize(0.01, 1)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(11.5, 4))
ax1.contour(xx, yy, prior, levels, cmap=cmap, norm=norm, zorder=-20)
ax1.contourf(xx, yy, prior, levels, cmap=cmap, norm=norm, zorder=-10)
ax1.set_title("Prior distribution")
ax2.contour(xx, yy, likelihood, levels, cmap=cmap, norm=norm, zorder=-20)
ax2.contourf(xx, yy, likelihood, levels, cmap=cmap, norm=norm, zorder=-10)
ax2.set_title("Likelihood function")
ax3.contour(xx, yy, posterior, levels, cmap=cmap, norm=norm, zorder=-20)
ax3.contourf(xx, yy, posterior, levels, cmap=cmap, norm=norm, zorder=-10)
ax3.set_title("Posterior distribution")
for ax in [ax1, ax2, ax3]:
    ax.set_xticks([])
    ax.set_yticks([])
fig.tight_layout()
#fig.savefig("../tex/figures/bayes_theorem.pdf")

## Bayesian Linear Regression

In [ ]:
from regression import LinearRegression

def flatten(*args):
    return (arg.flatten() for arg in args)

In [ ]:
np.random.seed(2)
cov = 0.2**2
x = np.vstack([
        np.random.normal(1.8, size=[6,1]),
        np.random.normal(-1.8, size=[6,1])
        ])
y = np.sin(x) + np.random.normal(scale=np.sqrt(cov), size=x.shape)
x_ref = np.linspace(-5, 5, 100).reshape(-1, 1)

bases = [
        lambda x: np.hstack([1., x, x**2, x**3, x**4, x**5]),
        lambda x: np.hstack([1., x]),
        lambda x: np.hstack([np.exp(-0.5*(x-μ)**2) for μ in range(-3, 4)])
        ]
alphas = [0.1, 0.1, 1]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 2.5))
for ax, basis, alpha in zip([ax1, ax2, ax3], bases, alphas):
    l = LinearRegression(basis, β=1/cov, α=alpha)
    l.fit(x, y)
    yy, std = flatten(*l.predict(x_ref, samples="std"))
    ax.plot(x_ref, yy, color="k", linewidth=2, zorder=-10)
    ax.fill_between(x_ref.flatten(), yy-std, yy+std, color="#BBBBBB", zorder=-40)
    ax.fill_between(x_ref.flatten(), yy-std*3, yy+std*3, color="#E0E0E0", zorder=-50)
    ax.scatter(*flatten(x, y), 40, edgecolor="#000000", facecolor="#FFFFFF", linewidth=1.2)
    ax.set_xlim(-5, 5)
    ax.set_ylim(-2, 2)
    ax.set_yticks([-2, -1, 0, 1, 2])
    ax.set_yticklabels(["-2", "-1", "0", "1", "2"])
    ax.label_outer()
fig.tight_layout()
#fig.savefig("../tex/figures/bayesian_regression.pdf")

## Absorption in the Microwave Region

In [ ]:
import mwrt
import formulas as fml

In [ ]:
νs = np.linspace(10, 80, 1000)
T = 273.15
θ = 300/T
p = 850.
rh = 100.
esat = fml.esat(T=T)
e = fml.e(esat=esat, RH=rh)
qliq = 0.0001

ylim = 8.0e-7, 1.0e-2
hatpro_o2 = np.array([22.24, 23.04, 23.84, 25.44, 26.24, 27.84, 31.40])
hatpro_hu = np.array([51.26, 52.28, 53.86, 54.94, 56.66, 57.30, 58.00])

def as_absorp(f):
    def absorp(ν, *args, **kwargs):
        return 4*np.pi*ν*1.0e9/299792458.*np.imag(f(ν, *args, **kwargs))
    return absorp

gas_absorp = as_absorp(mwrt.liebe93.refractivity_gaseous)
h2o_absorp = as_absorp(mwrt.liebe93.refractivity_H2O)
cld_absorp = as_absorp(mwrt.tkc.refractivity_lwc)

α_gas = gas_absorp(νs, θ, p-e, e)
α_h2o = h2o_absorp(νs, θ, p-e, e)
α_dry = α_gas - α_h2o
α_cld = qliq * fml.ρ(T=T, p=p, e=e) * cld_absorp(νs, θ)

α_ho2 = gas_absorp(hatpro_o2, θ, p-e, e) + qliq * fml.ρ(T=T, p=p, e=e) * cld_absorp(hatpro_o2, θ)
α_hhu = gas_absorp(hatpro_hu, θ, p-e, e) + qliq * fml.ρ(T=T, p=p, e=e) * cld_absorp(hatpro_hu, θ)

fig, ax = plt.subplots(1, 1, figsize=(8, 3.5))
ax.semilogy(νs, α_gas + α_cld, color="#000000", linewidth=2.5, zorder=-10, label="total")
ax.semilogy(νs, α_cld, linewidth=1.5, color="#666666", zorder=-50, label="cloud")
ax.semilogy(νs, α_dry, linewidth=1.5, color="#33a02c", zorder=-40, label="dry")
ax.semilogy(νs, α_h2o, linewidth=1.5, color="#1f78b4", zorder=-30, label="H₂O")
ax.scatter(hatpro_o2, α_ho2*1.55, 90, marker="|", zorder=-5, color="#000000")
ax.scatter(hatpro_o2, α_ho2*1.35, 20, marker="v", zorder=-5, color="#000000")
ax.scatter(hatpro_hu, α_hhu*1.55, 90, marker="|", zorder=-5, color="#000000")
ax.scatter(hatpro_hu, α_hhu*1.35, 20, marker="v", zorder=-5, color="#000000")

ax.legend(loc="upper left", ncol=2)
ax.text(24, 1.3e-4, "K band", ha="center", fontsize=12)
ax.text(60, 6.0e-4, "V band", ha="center", fontsize=12)

ax.set_xlabel("frequency [GHz]")
ax.set_ylabel("absorption [1/m]")

ax.set_ylim(*ylim)
ax.set_xlim(min(νs), max(νs))
fig.tight_layout()
#fig.savefig("../tex/figures/absorption.pdf")